In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brain-tumour/brain_tumor_dataset/no/34 no.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/N20.JPG
/kaggle/input/brain-tumour/brain_tumor_dataset/no/N1.JPG
/kaggle/input/brain-tumour/brain_tumor_dataset/no/49 no.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/N15.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/No18.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/31 no.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/no 6.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/3 no.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/no 5.jpeg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/no 1.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/26 no.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/N26.JPG
/kaggle/input/brain-tumour/brain_tumor_dataset/no/N5.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/6 no.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/No21.jpg
/kaggle/input/brain-tumour/brain_tumor_dataset/no/17 no.jp

In [4]:
import numpy as np
import pandas as pd
import kagglehub
import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D as GAP
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

2025-05-23 20:55:13.880418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748033714.061060      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748033714.111417      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
import os
import shutil
import random
from pathlib import Path

source_dir = Path("/kaggle/input/brain-tumour/brain_tumor_dataset")
output_dir = Path("/kaggle/working/dataset")

train_split = 0.7
val_split = 0.15
test_split = 0.15

classes = ['yes', 'no']
valid_extensions = ['.jpg', '.JPG', '.png']

for split in ['train', 'val', 'test']:
    for cls in classes:
        (output_dir / split / cls).mkdir(parents=True, exist_ok=True)

def split_data(class_name):
    src = source_dir / class_name
    all_files = [f for f in src.iterdir() if f.suffix.lower() in valid_extensions]
    random.shuffle(all_files)

    total = len(all_files)
    train_end = int(total * train_split)
    val_end = train_end + int(total * val_split)

    train_files = all_files[:train_end]
    val_files = all_files[train_end:val_end]
    test_files = all_files[val_end:]

    for files, split in zip([train_files, val_files, test_files], ['train', 'val', 'test']):
        for f in files:
            dest = output_dir / split / class_name / f.name
            shutil.copy(f, dest)

for cls in classes:
    split_data(cls)

print("✅ Dataset split into train/val/test successfully.")


✅ Dataset split into train/val/test successfully.


In [9]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


train_dir = '/kaggle/working/dataset/train'
val_dir = '/kaggle/working/dataset/val'
test_dir = '/kaggle/working/dataset/test'

IMG_SIZE = 224
BATCH_SIZE = 16

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),                    
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std)
])

In [10]:
train_dataset = datasets.ImageFolder(root='dataset/train', transform=transform)
val_dataset   = datasets.ImageFolder(root='dataset/val', transform=transform)
test_dataset  = datasets.ImageFolder(root='dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print("Classes:", train_dataset.classes)
print("Label mapping:", train_dataset.class_to_idx)

Classes: ['no', 'yes']
Label mapping: {'no': 0, 'yes': 1}


In [11]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'resnet50'

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 205MB/s]


In [12]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.sigmoid(outputs).squeeze() > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

Epoch [1/20], Loss: 6.7927
Validation Accuracy: 83.33%
Epoch [2/20], Loss: 5.2782
Validation Accuracy: 83.33%
Epoch [3/20], Loss: 3.9422
Validation Accuracy: 80.56%
Epoch [4/20], Loss: 3.3304
Validation Accuracy: 80.56%
Epoch [5/20], Loss: 2.4583
Validation Accuracy: 80.56%
Epoch [6/20], Loss: 1.8839
Validation Accuracy: 77.78%
Epoch [7/20], Loss: 1.2790
Validation Accuracy: 77.78%
Epoch [8/20], Loss: 1.3258
Validation Accuracy: 77.78%
Epoch [9/20], Loss: 0.7078
Validation Accuracy: 83.33%
Epoch [10/20], Loss: 0.8280
Validation Accuracy: 88.89%
Epoch [11/20], Loss: 0.6016
Validation Accuracy: 80.56%
Epoch [12/20], Loss: 0.4277
Validation Accuracy: 80.56%
Epoch [13/20], Loss: 0.3093
Validation Accuracy: 80.56%
Epoch [14/20], Loss: 0.5749
Validation Accuracy: 83.33%
Epoch [15/20], Loss: 0.3832
Validation Accuracy: 83.33%
Epoch [16/20], Loss: 0.2208
Validation Accuracy: 83.33%
Epoch [17/20], Loss: 0.2491
Validation Accuracy: 83.33%
Epoch [18/20], Loss: 0.1855
Validation Accuracy: 83.33%
E

In [13]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.sigmoid(outputs).squeeze() > 0.5
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"🧪 Test Accuracy: {100 * correct / total:.2f}%")

🧪 Test Accuracy: 84.62%


In [14]:
import torch
import torch.nn as nn
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = 'resnet18'

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 233MB/s]


In [15]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.sigmoid(outputs).squeeze() > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

Epoch [1/20], Loss: 6.5383
Validation Accuracy: 50.00%
Epoch [2/20], Loss: 4.9966
Validation Accuracy: 69.44%
Epoch [3/20], Loss: 4.0221
Validation Accuracy: 77.78%
Epoch [4/20], Loss: 3.3540
Validation Accuracy: 83.33%
Epoch [5/20], Loss: 2.6752
Validation Accuracy: 83.33%
Epoch [6/20], Loss: 2.3800
Validation Accuracy: 83.33%
Epoch [7/20], Loss: 1.9552
Validation Accuracy: 86.11%
Epoch [8/20], Loss: 1.5375
Validation Accuracy: 86.11%
Epoch [9/20], Loss: 1.5739
Validation Accuracy: 88.89%
Epoch [10/20], Loss: 1.0515
Validation Accuracy: 88.89%
Epoch [11/20], Loss: 1.1590
Validation Accuracy: 91.67%
Epoch [12/20], Loss: 1.0442
Validation Accuracy: 86.11%
Epoch [13/20], Loss: 0.8324
Validation Accuracy: 88.89%
Epoch [14/20], Loss: 0.5734
Validation Accuracy: 91.67%
Epoch [15/20], Loss: 0.6386
Validation Accuracy: 91.67%
Epoch [16/20], Loss: 0.5580
Validation Accuracy: 91.67%
Epoch [17/20], Loss: 0.4984
Validation Accuracy: 91.67%
Epoch [18/20], Loss: 0.3624
Validation Accuracy: 91.67%
E

In [18]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.sigmoid(outputs).squeeze() > 0.5
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"🧪 Test Accuracy: {100 * correct / total:.2f}%")

🧪 Test Accuracy: 84.62%


In [19]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 1)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [20]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            preds = torch.sigmoid(outputs).squeeze() > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")


Epoch [1/20], Loss: 6.4278
Validation Accuracy: 80.56%
Epoch [2/20], Loss: 4.3366
Validation Accuracy: 63.89%
Epoch [3/20], Loss: 2.3491
Validation Accuracy: 69.44%
Epoch [4/20], Loss: 1.2533
Validation Accuracy: 88.89%
Epoch [5/20], Loss: 0.5307
Validation Accuracy: 83.33%
Epoch [6/20], Loss: 0.3751
Validation Accuracy: 88.89%
Epoch [7/20], Loss: 0.2691
Validation Accuracy: 77.78%
Epoch [8/20], Loss: 0.1304
Validation Accuracy: 75.00%
Epoch [9/20], Loss: 0.1531
Validation Accuracy: 86.11%
Epoch [10/20], Loss: 0.0413
Validation Accuracy: 88.89%
Epoch [11/20], Loss: 0.0682
Validation Accuracy: 86.11%
Epoch [12/20], Loss: 0.0452
Validation Accuracy: 86.11%
Epoch [13/20], Loss: 0.0403
Validation Accuracy: 86.11%
Epoch [14/20], Loss: 0.0715
Validation Accuracy: 86.11%
Epoch [15/20], Loss: 0.0830
Validation Accuracy: 86.11%
Epoch [16/20], Loss: 0.0524
Validation Accuracy: 86.11%
Epoch [17/20], Loss: 0.0171
Validation Accuracy: 86.11%
Epoch [18/20], Loss: 0.0842
Validation Accuracy: 86.11%
E

In [21]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.sigmoid(outputs).squeeze() > 0.5
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 71.79%
